<a href="https://colab.research.google.com/github/etoshey/colabC3D/blob/main/ScaleTool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Opensim Sclaing

In [ ]:
#@title Install MiniConda (~40s)
import sys
import os

# ensure cwd is /content
os.chdir("/content")

# Install MiniConda
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.3-Linux-x86_64.sh
!bash ./Miniconda3-py37_4.8.3-Linux-x86_64.sh -b -f -p /usr/local

# Add the path to the conda package installation location
sys.path.append('/usr/local/lib/python3.7/site-packages')

In [ ]:
#@title Install opensim with Conda (~1min)

# Install Opensim
!conda install -c opensim-org opensim -y
import opensim as osim

In [64]:
#@title Importing Misc Libraries
from google.colab import files #to upload user's files
import numpy as np #used in some functions
import time
!pip install xmltodict


In [65]:
#@title Step1 : upload static trc file
import ipywidgets as widgets
from google.colab import files



file_name = ""

upload_btn = widgets.Button(description='Upload')
def upload_btn_eventhandler(obj):

  global file_name
  uploaded = files.upload()
  file_name = list(uploaded.keys())[0]

  print('Upload Completed ==> '+file_name)


upload_btn.on_click(upload_btn_eventhandler)
display(upload_btn)



Button(description='Upload', style=ButtonStyle())

Saving subject01_static.trc to subject01_static.trc
Upload Completed ==> subject01_static.trc


In [85]:
#@title OpenSim SclaeTool Form
import os
import ipywidgets as widgets
import xmltodict
import math


class class_static_markers:
  def __init__(self,name,x,y,z,body=''):
    self.name = name
    self.x = x
    self.y = y
    self.z = z
    self.body = body



static_markers_arr = []
static_markers = []
list_markerset_name = []
_list = [[],[],[],[]]

model_body = []

model_name=''
#get all files
all_xml_files =[]
all_osim_files =[]
all_trc_files =['select..']
for file_path in os.listdir('/content/'):
    if os.path.isfile(os.path.join('/content/', file_path)):
      if os.path.splitext(file_path)[1]=='.xml':
        all_xml_files.append(file_path)
      elif os.path.splitext(file_path)[1]=='.osim':
        all_osim_files.append(file_path)
      elif os.path.splitext(file_path)[1]=='.trc':
        all_trc_files.append(file_path)



#--------------------------------------------------------------------------------------------------------#
all_trc_dropdown = widgets.Dropdown(description='Selecte static data(trc):',options = all_trc_files)
select_trc_button = widgets.Button(
    description='OK',
    button_style='info'
)

def dropdown_handler(change):
    global static_trc_path
    static_trc_path = change.new
all_trc_dropdown.observe(dropdown_handler, names='value')


def select_trc_btn_eventhandler(obj):
  global static_markers_arr,static_markers,static_trc_path
  my_file = open(static_trc_path, "r")
  my_file.readline();#line1
  my_file.readline();#line2
  my_file.readline();#line3
  param = my_file.readline();
  my_file.readline();
  my_file.readline();
  param2 = my_file.readline();

  static_markers = param.split('\t')
  static_markers = [item for item in static_markers if item != '' and item != '\n']
  static_markers = static_markers[2:]

  marker_pos = param2.split('\t');
  marker_pos = [item for item in marker_pos if item != '' and item != '\n']
  marker_pos = marker_pos[2:]


  for inx,m in enumerate(static_markers):
    static_markers_arr.append(class_static_markers(m,marker_pos[(inx*3)] , marker_pos[(inx*3)+1] , marker_pos[(inx*3)+2]))

  show_form()


select_trc_button.on_click(select_trc_btn_eventhandler)

#--------------------------------------------------------------------------------------------------------#

h0 = widgets.HBox([all_trc_dropdown, select_trc_button ])
display(h0)

#--------------------------------------------------------------------------------------------------------#


def show_form():
  #-------> Section 1
  section1_lbl =  widgets.Label(value="----------------------------- Anthropometry & Model -----------------------------");
  mass = widgets.Text(value='76',placeholder='Type something',description='Weight(Kg):',disabled=False)
  height = widgets.Text(value='1800',placeholder='Type something',description='height(Cm):',disabled=False)

  #-------> GenericModelMaker
  model_dropdown = widgets.Dropdown(description='Model:',options=['','gait2392_simbody', 'gait2354_simbody','gait10dof18musc'])
  model_dropdown.observe(dropdown_Selectmodel, names='value')
    #Generate markerSet.xml based on static trc



  spliter_lbl =  widgets.Label(value="-------------------------------------------------------------------------------------------------------------------");
  #marker_file
  marker_file = widgets.Label(value="static trial : " + static_trc_path)
  #MarkerSet
  MarkerSet_dropdown = widgets.Dropdown(description='MarkerSet :',options = ['Plug-in Gait' , 'Cast' , 'Cleveland'])
  MarkerSet_dropdown.observe(dropdown_MarkerSet, names='value')



  h1 = widgets.HBox([mass, height , model_dropdown])
  h2 = widgets.VBox([marker_file,MarkerSet_dropdown])

  display(section1_lbl,h1,spliter_lbl,h2)




def Setup_Xml_Generator():
  print("ss")


def update_scale_factor(change):
  print(change.new)
  print([x for x in static_markers_arr if x.name == change.new])


def show_model_scaler():
    #-------> ModelScaler
  ModelScaler_lbl =  widgets.Label(value="----------------------------- ModelScaler -----------------------------");
  # torso
  torso1_marker_dropdown = widgets.Dropdown(description='Torso 1:',options = static_markers)
  torso1_marker_dropdown.observe(update_scale_factor, names='value')
  torso2_marker_dropdown = widgets.Dropdown(description='Torso 2:',options = static_markers)
  torso2_marker_dropdown.observe(update_scale_factor, names='value')
  torso3_marker_dropdown = widgets.Dropdown(description='Torso 3:',options = static_markers)
  torso_scale = widgets.Text(value='',placeholder='Type something',description='Sclae Factor:')


  model_body
  #pelvis
  pelvis1_marker_dropdown = widgets.Dropdown(description='Pelvis 1:',options = static_markers)
  pelvis2_marker_dropdown = widgets.Dropdown(description='Pelvis 2:',options = static_markers)
  pelvis_scale = widgets.Text(value='',placeholder='Type something',description='Sclae Factor:')
  #thigh
  Lthigh1_marker_dropdown = widgets.Dropdown(description='Left thigh1:',options = static_markers)
  Lthigh2_marker_dropdown = widgets.Dropdown(description='Left thigh2:',options = static_markers)
  Lthigh_scale = widgets.Text(value='',placeholder='Type something',description='Sclae Factor:')

  Rthigh1_marker_dropdown = widgets.Dropdown(description='Right thigh1:',options = static_markers)
  Rthigh2_marker_dropdown = widgets.Dropdown(description='Right thigh2:',options = static_markers)
  Rthigh_scale = widgets.Text(value='',placeholder='Type something',description='Sclae Factor:')
  #shank
  Lshank1_marker_dropdown = widgets.Dropdown(description='Left shank1:',options = static_markers)
  Lshank2_marker_dropdown = widgets.Dropdown(description='Left shank2:',options = static_markers)
  Lshank_scale = widgets.Text(value='',placeholder='Type something',description='Sclae Factor:')

  Rshank1_marker_dropdown = widgets.Dropdown(description='Right shank1:',options = static_markers)
  Rshank2_marker_dropdown = widgets.Dropdown(description='Right shank2:',options = static_markers)
  Rshank_scale = widgets.Text(value='',placeholder='Type something',description='Sclae Factor:')
  #foot
  Lfoot1_marker_dropdown = widgets.Dropdown(description='Left foot1:',options = static_markers)
  Lfoot2_marker_dropdown = widgets.Dropdown(description='Left foot2:',options = static_markers)
  Lfoot_scale = widgets.Text(value='',placeholder='Type something',description='Sclae Factor:')

  Rfoot1_marker_dropdown = widgets.Dropdown(description='Right foot1:',options = static_markers)
  Rfoot2_marker_dropdown = widgets.Dropdown(description='Right foot2:',options = static_markers)
  Rfoot_scale = widgets.Text(value='',placeholder='Type something',description='Sclae Factor:')

  h3 = widgets.HBox([torso1_marker_dropdown,torso2_marker_dropdown,torso_scale])
  h4 = widgets.HBox([pelvis1_marker_dropdown,pelvis2_marker_dropdown,pelvis_scale])
  h5 = widgets.HBox([Lthigh1_marker_dropdown,Lthigh2_marker_dropdown,Lthigh_scale])
  h6 = widgets.HBox([Rthigh1_marker_dropdown,Rthigh2_marker_dropdown,Rthigh_scale])
  h7 = widgets.HBox([Lshank1_marker_dropdown,Lshank2_marker_dropdown,Lshank_scale])
  h8 = widgets.HBox([Rshank1_marker_dropdown,Rshank2_marker_dropdown,Rshank_scale])
  h9 = widgets.HBox([Lfoot1_marker_dropdown,Lfoot2_marker_dropdown,Lfoot_scale])
  h10 = widgets.HBox([Rfoot1_marker_dropdown,Rfoot2_marker_dropdown,Rfoot_scale])

  display(ModelScaler_lbl,h3,h4,h5,h6,h7,h8,h9,h10)





def Compute_Scale_Factor(model_p1,model_p2,data_p1,data_p2):
  model_dis_x = model_p1[0] - model_p2[0]
  model_dis_y = model_p1[1] - model_p2[1]
  model_dis_z = model_p1[2] - model_p2[2]
  model_len = math.sqrt(model_dis_x*model_dis_x + model_dis_y * model_dis_y + model_dis_z * model_dis_z)

  data_dis_x = data_p1[0] - data_p2[0]
  data_dis_y = data_p1[1] - data_p2[1]
  data_dis_z = data_p1[2] - data_p2[2]
  data_len = math.sqrt(data_dis_x * data_dis_x + data_dis_y * data_dis_y + data_dis_z * data_dis_z)

  print(model_len , ' - ', data_len)




def show_scaler_section(obj):

  #Make table
  for i in range(4):
    for j in range(len(_list[i])):
      print(_list[i][j].description , ' - ',_list[i][j].value)

  #read model .osim
  print(model_name)
  if model_name =='gait2392_simbody':
    !wget -q https://raw.githubusercontent.com/etoshey/colabC3D/main/Files/Opensim/Model/gait2392_simbody.osim
  elif model_name =='gait2354_simbody':
    !wget -q https://raw.githubusercontent.com/etoshey/colabC3D/main/Files/Opensim/Model/gait2354_simbody.osim


  xml_data = open(model_name+'.osim', 'r').read()  # Read data
  xmlDict = xmltodict.parse(xml_data)
  M_list = xmlDict['OpenSimDocument']['Model']['BodySet']['objects']['Body']
  for item in M_list:
    model_body.append(item['@name'])

  show_model_scaler()


def dropdown_Selectmodel(change):
  global model_name
  model_name = change.new


def dropdown_MarkerSet(change):
  global static_trc_path

  xml_file_name = ""
  match change.new:
    case "Cleveland":
      if os.path.isfile("/content/gait2392_Scale_MarkerSet.xml") == False:
        !wget -q https://raw.githubusercontent.com/etoshey/colabC3D/main/Files/Opensim/MarkerSet/gait2392_Scale_MarkerSet.xml
      xml_file_name = "gait2392_Scale_MarkerSet.xml"

  print(xml_file_name)
  if xml_file_name != "":
    #read xml file
    xml_data = open(xml_file_name, 'r').read()  # Read data
    xmlDict = xmltodict.parse(xml_data)
    M_list = xmlDict['OpenSimDocument']['MarkerSet']['objects']['Marker']


    for i in range(0, len(M_list),4):


      if (len(M_list) - i >= 4):
        for j in range(i,i+4):
          _list[j-i].append(widgets.Dropdown(description=M_list[j]['@name'] +':',options = static_markers , value=static_markers[find_index(static_markers,M_list[j]['@name'])]))
          pos = M_list[j]['location'].split(' ')
          pos = [item for item in pos if item != '' and item != '\n']

          list_markerset_name.append(class_static_markers(M_list[j]['@name'],pos[0],pos[1],pos[2],M_list[j]['body']))
      else:
        for j in range(i,len(M_list)):
          _list[j-i].append(widgets.Dropdown(description=M_list[j]['@name'] +':',options = static_markers , value=static_markers[find_index(static_markers,M_list[j]['@name'])]))
          pos = M_list[j]['location'].split(' ')
          pos = [item for item in pos if item != '' and item != '\n']
          list_markerset_name.append(class_static_markers(M_list[j]['@name'],pos[0],pos[1],pos[2],M_list[j]['body']))



    # make table
    V1 = widgets.VBox(_list[0])
    V2 = widgets.VBox(_list[1])
    V3 = widgets.VBox(_list[2])
    V4 = widgets.VBox(_list[3])

    H = widgets.HBox([V1,V2,V3,V4])
    Title =  widgets.Label(value="Label Matching ==>");
    #Next
    Next_button = widgets.Button(description='Next Step',button_style='success')
    Next_button.on_click(show_scaler_section)
    display(Title,H,Next_button)




def find_index(_list,p):
  try:
    return _list.index(p)
  except ValueError:
    return 0



def show_proc_btn():
  #Processing
  run_button = widgets.Button(description='Run!',button_style='success')
  display(run_button)


Label(value='----------------------------- Anthropometry & Model -----------------------------')

Label(value='-------------------------------------------------------------------------------------------------…

gait2392_Scale_MarkerSet.xml


Label(value='Label Matching ==>')

Button(button_style='success', description='Next Step', style=ButtonStyle())

Sternum:  -  Sternum
R.ASIS:  -  R.ASIS
R.Thigh.Front:  -  R.Thigh.Front
R.Shank.Upper:  -  R.Shank.Upper
R.Ankle.Med:  -  R.Ankle.Med
R.Toe.Lat:  -  R.Toe.Lat
L.Thigh.Front:  -  L.Thigh.Front
L.Shank.Upper:  -  L.Shank.Upper
L.Ankle.Med:  -  L.Ankle.Med
L.Toe.Lat:  -  L.Toe.Lat
R.Acromium:  -  R.Acromium
L.ASIS:  -  L.ASIS
R.Thigh.Rear:  -  R.Thigh.Rear
R.Shank.Front:  -  R.Shank.Front
R.Heel:  -  R.Heel
R.Toe.Med:  -  R.Toe.Med
L.Thigh.Rear:  -  L.Thigh.Rear
L.Shank.Front:  -  L.Shank.Front
L.Heel:  -  L.Heel
L.Toe.Med:  -  L.Toe.Med
L.Acromium:  -  L.Acromium
V.Sacral:  -  V.Sacral
R.Knee.Lat:  -  R.Knee.Lat
R.Shank.Rear:  -  R.Shank.Rear
R.Midfoot.Sup:  -  R.Midfoot.Sup
R.Toe.Tip:  -  R.Toe.Tip
L.Knee.Lat:  -  L.Knee.Lat
L.Shank.Rear:  -  L.Shank.Rear
L.Midfoot.Sup:  -  L.Midfoot.Sup
L.Toe.Tip:  -  L.Toe.Tip
Top.Head:  -  Top.Head
R.Thigh.Upper:  -  R.Thigh.Upper
R.Knee.Med:  -  R.Knee.Med
R.Ankle.Lat:  -  R.Ankle.Lat
R.Midfoot.Lat:  -  R.Midfoot.Lat
L.Thigh.Upper:  -  L.Thigh.Uppe

Label(value='----------------------------- ModelScaler -----------------------------')

In [ ]:
from lxml import etree

# xml_data = open('gait2392_Scale_MarkerSet.xml', 'r').read()  # Read data



# print(xml_data)


xml_data = open('gait2392_Scale_MarkerSet.xml', 'r').read()  # Read data
xmlDict = xmltodict.parse(xml_data)
M_list = xmlDict['OpenSimDocument']['MarkerSet']['objects']['Marker']
list_marker_name = []
for m in M_list:
  list_marker_name.append(m['@name'])

print(list_marker_name)





['Sternum', 'R.Acromium', 'L.Acromium', 'Top.Head', 'R.ASIS', 'L.ASIS', 'V.Sacral', 'R.Thigh.Upper', 'R.Thigh.Front', 'R.Thigh.Rear', 'R.Knee.Lat', 'R.Knee.Med', 'R.Shank.Upper', 'R.Shank.Front', 'R.Shank.Rear', 'R.Ankle.Lat', 'R.Ankle.Med', 'R.Heel', 'R.Midfoot.Sup', 'R.Midfoot.Lat', 'R.Toe.Lat', 'R.Toe.Med', 'R.Toe.Tip', 'L.Thigh.Upper', 'L.Thigh.Front', 'L.Thigh.Rear', 'L.Knee.Lat', 'L.Knee.Med', 'L.Shank.Upper', 'L.Shank.Front', 'L.Shank.Rear', 'L.Ankle.Lat', 'L.Ankle.Med', 'L.Heel', 'L.Midfoot.Sup', 'L.Midfoot.Lat', 'L.Toe.Lat', 'L.Toe.Med', 'L.Toe.Tip']


In [77]:

pelvis1_marker_dropdown = widgets.Dropdown(description='Pelvis 1:',options = ['static_markers','rrrr'])
display(pelvis1_marker_dropdown)

Dropdown(description='Pelvis 1:', options=('static_markers', 'rrrr'), value='static_markers')

In [83]:
print(_list[0][0].value)

R.Shank.Front
